## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os.path
sys.path.append("..")

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [30]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Atuona,PF,-9.8000,-139.0333,25.78,72,15,7.78,few clouds
1,Savonlinna,FI,61.8699,28.8800,-21.97,84,90,1.03,overcast clouds
2,Flinders,AU,-34.5833,150.8552,21.78,85,59,4.36,broken clouds
3,Punta Arenas,CL,-53.1500,-70.9167,17.06,36,75,13.38,broken clouds
4,Chuy,UY,-33.6971,-53.4616,25.85,57,80,3.31,broken clouds


In [31]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? -15
What is the maximum temperature you would like for your trip? -2


In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,Nikolskoye,RU,59.7035,30.7861,-11.78,96,100,2.13,light snow
31,Tasiilaq,GL,65.6145,-37.6368,-3.91,60,100,3.61,overcast clouds
38,Puqi,CN,29.7167,113.8833,-2.10,84,100,5.36,light snow
44,Ketchikan,US,55.3422,-131.6461,-8.01,35,1,3.60,clear sky
46,Torbay,CA,47.6666,-52.7314,-2.63,87,90,9.26,light shower snow


In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   75
Country                75
Lat                    75
Lng                    75
Max Temp               75
Humidity               75
Cloudiness             75
Wind Speed             75
Current Description    75
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City                   75
Country                75
Lat                    75
Lng                    75
Max Temp               75
Humidity               75
Cloudiness             75
Wind Speed             75
Current Description    75
dtype: int64

In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Nikolskoye,RU,-11.78,light snow,59.7035,30.7861,
31,Tasiilaq,GL,-3.91,overcast clouds,65.6145,-37.6368,
38,Puqi,CN,-2.10,light snow,29.7167,113.8833,
44,Ketchikan,US,-8.01,clear sky,55.3422,-131.6461,
46,Torbay,CA,-2.63,light shower snow,47.6666,-52.7314,
53,Kenai,US,-7.73,scattered clouds,60.5544,-151.2583,
55,Ilulissat,GL,-10.99,clear sky,69.2167,-51.1000,
82,Guozhen,CN,-4.12,overcast clouds,34.3659,107.3590,
101,Kurilsk,RU,-2.51,light snow,45.2333,147.8833,
114,Longyearbyen,SJ,-7.09,few clouds,78.2186,15.6401,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
    

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Nikolskoye,RU,-11.78,light snow,59.7035,30.7861,Tourist House - Sablino
31,Tasiilaq,GL,-3.91,overcast clouds,65.6145,-37.6368,The Red House
38,Puqi,CN,-2.10,light snow,29.7167,113.8833,Ramada Plaza Chibi
44,Ketchikan,US,-8.01,clear sky,55.3422,-131.6461,Snow's Cove Lodge
46,Torbay,CA,-2.63,light shower snow,47.6666,-52.7314,Gallows Cove


In [38]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))